# DealInfos_Word2Vec feature를 ES에 벌크 인서트

-  환경: ElasticSearch 6
-  테스트 머신: 장민 PC

```
PUT /dealinfos
{
mapping_dealinfos_word2vec.json
}
```


In [12]:
import requests
import time
import urllib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import json
import elasticsearch
import csv
import pickle
from elasticsearch.helpers import bulk
import re
import glob
import os
from datetime import timezone, timedelta, datetime
from pymongo import MongoClient
import random
import numpy as np
import sys
import pandas as pd

In [3]:
def make_documents(file_path):
    pred_dic = {}
    with open(file_path, 'rb') as jsonfile:
        for row in jsonfile:
            source_ = eval(row)
            source = {}
            source['v'] = source_['v']
            source['words'] = source_['words']
            source['values'] = source_['vectorizedWords']['values']
            doc =  {
                "_op_type":'create',
                "_index":"deal_word2vec",
                "_type":"doc",
                "_id": source['v'],
                "_source": source
            }
            yield(doc)

In [4]:
### Jangmin's PC
es_url = 'localhost:9200'

In [6]:
es = elasticsearch.Elasticsearch(es_url)

In [45]:
es.indices.get_mapping(index='deal_word2vec')

{'deal_word2vec': {'mappings': {'doc': {'properties': {'v': {'type': 'long'},
     'values': {'type': 'double'},
     'words': {'type': 'keyword'}}}}}}

# bulk inserting to ES

In [8]:
data_dir = r'/Users/jangmino/tensorflow/work/wepick_data/w2vec.json'
start = time.time()
total_time = 0
bulk(es, make_documents(data_dir))
total_time += (time.time() - start)
print("Elapsed Time {} sec".format(total_time))

Elapsed Time 1301.6904072761536 sec


In [9]:
res = es.search(index='deal_word2vec', body={'query': {'match':{'v':'2668602'}}})

In [10]:
res['hits']['total']

1

In [11]:
res


{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '2668602',
    '_index': 'deal_word2vec',
    '_score': 1.0,
    '_source': {'v': 2668602,
     'values': [-1.7802355289459229,
      18.870119285583495,
      2.5163070142269137,
      -10.703756618499757,
      1.600479006767273,
      3.5656559824943543,
      -5.844938635826111,
      6.633014583587647,
      8.395471286773683,
      -23.7651406288147,
      14.762410998344421,
      10.690511178970338,
      8.746411949396133,
      28.675686740875246,
      8.49290874004364,
      -3.034431028366089,
      36.52057704925537,
      38.59238967895508,
      -3.684904098510742,
      33.746253967285156,
      -8.964104914665223,
      -15.591403150558472,
      -0.33317070007324223,
      30.87879137992859,
      9.32277364730835,
      29.246194028854372,
      -28.27330641746521,
      -20.553472375869752,
      -11.5706449508667,
      -4.212529945373535,
      -10.324701023101808,
  

### wepick_data

format:
deal_id, user_id, slot_id, timestamp, impression_time, label(always one)

In [14]:
data_path = r'/Users/jangmino/tensorflow/work/wepick_data/pos_dataset_small.csv'
df = pd.read_csv(data_path, header=None, names=["v", "u", "slot", "rgtime", "imp_dt", "label"])

In [58]:
df['

TypeError: cannot convert the series to <class 'int'>

In [52]:
def make_wepick_documents(df):
    for u, hist in df.groupby('u'):
        source = {}
        vec = []
        source['u'] = u
        for i in range(len(hist.index)):
            data = dict(hist.iloc[i])
            #data['rgtime'] = datetime.fromtimestamp(data['rgtime'])
            del data['u']
            del data['label']
            vec.append(data)
        source['hist'] = vec
        doc = {
            "_op_type":'create',
            "_index":"wepick_seq",
            "_type":"doc",
            "_id": u,
            "_source": source
        }
        yield(doc)

In [53]:
for x in make_wepick_documents(df):
    print(x)
    break

{'_op_type': 'create', '_index': 'wepick_seq', '_type': 'doc', '_id': 1000006, '_source': {'u': 1000006, 'hist': [{'v': 3398131, 'slot': 22, 'rgtime': 1520769276608, 'imp_dt': '2018-03-11 20'}, {'v': 3365865, 'slot': 5, 'rgtime': 1520830230903, 'imp_dt': '2018-03-12 13'}, {'v': 3284877, 'slot': 6, 'rgtime': 1520830251361, 'imp_dt': '2018-03-12 13'}, {'v': 3384031, 'slot': 9, 'rgtime': 1520830278597, 'imp_dt': '2018-03-12 13'}, {'v': 3398754, 'slot': 1, 'rgtime': 1520830417854, 'imp_dt': '2018-03-12 13'}, {'v': 3387193, 'slot': 2, 'rgtime': 1520850826955, 'imp_dt': '2018-03-12 19'}, {'v': 3429250, 'slot': 1, 'rgtime': 1521384018543, 'imp_dt': '2018-03-18 23'}, {'v': 3337685, 'slot': 10, 'rgtime': 1521384179454, 'imp_dt': '2018-03-18 23'}, {'v': 3418856, 'slot': 10, 'rgtime': 1521541700613, 'imp_dt': '2018-03-20 19'}, {'v': 3339612, 'slot': 7, 'rgtime': 1522072104891, 'imp_dt': '2018-03-26 22'}, {'v': 1938495, 'slot': 4, 'rgtime': 1522215978153, 'imp_dt': '2018-03-28 14'}, {'v': 3472717,

In [54]:
es.indices.get_mapping(index='wepick_seq')

{'wepick_seq': {'mappings': {}}}

In [55]:
start = time.time()
total_time = 0
bulk(es, make_wepick_documents(df))
total_time += (time.time() - start)
print("Elapsed Time {} sec".format(total_time))

SerializationError: ({'u': 1000006, 'hist': [{'v': 3398131, 'slot': 22, 'rgtime': 1520769276608, 'imp_dt': '2018-03-11 20'}, {'v': 3365865, 'slot': 5, 'rgtime': 1520830230903, 'imp_dt': '2018-03-12 13'}, {'v': 3284877, 'slot': 6, 'rgtime': 1520830251361, 'imp_dt': '2018-03-12 13'}, {'v': 3384031, 'slot': 9, 'rgtime': 1520830278597, 'imp_dt': '2018-03-12 13'}, {'v': 3398754, 'slot': 1, 'rgtime': 1520830417854, 'imp_dt': '2018-03-12 13'}, {'v': 3387193, 'slot': 2, 'rgtime': 1520850826955, 'imp_dt': '2018-03-12 19'}, {'v': 3429250, 'slot': 1, 'rgtime': 1521384018543, 'imp_dt': '2018-03-18 23'}, {'v': 3337685, 'slot': 10, 'rgtime': 1521384179454, 'imp_dt': '2018-03-18 23'}, {'v': 3418856, 'slot': 10, 'rgtime': 1521541700613, 'imp_dt': '2018-03-20 19'}, {'v': 3339612, 'slot': 7, 'rgtime': 1522072104891, 'imp_dt': '2018-03-26 22'}, {'v': 1938495, 'slot': 4, 'rgtime': 1522215978153, 'imp_dt': '2018-03-28 14'}, {'v': 3472717, 'slot': 4, 'rgtime': 1522595745807, 'imp_dt': '2018-04-02 00'}, {'v'